# Instructions of use

This notebook is used as a demonstration of how model parameters that are not in the inputs.xlsx file can be modified.
There are two ways:
    1.  running the model as usual and adding the names of the parameters to update as follows:
     
         python run.py land_productivity_biofuels_marg_ej_mha=1
     
         In this case, only constants and lookups can be modified. Pandas time-series can also be passed in this way.
         
         e.g. python run.py -x BAU -p land_productivity_biofuels_marg_ej_mha=66 historic_water_use_by_type_sectors_subelectricity_gas_and_water_supply_subgreen_water="functions.lookup(x, [1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])"
     
     
    2. Running the model from a python script in which the model components are modified (see section Running simulations updating xarrays, lookups and constants)

## IMPORTANT NOTES

Please take into account that the default scenario is BAU. If you want to use a different scenario, please modify the 'scenario_sheet' value of the config dictionary to the name of the scenario you wish to run.

If the load_model fails, you are not using the correct kernel.

From terminal, run the following comand:

    ipython kernel install --user --name=pymedeas
    
This will create a new kernel, pointing to the right virtual environment.

Then you only have to go to the "kernel" menu and from the "change_kernel" menu select "pymedeas.

# The following 3 cells need to be run in order to load the model in memory

In [ ]:
from pytools.tools import update_paths, load_model
from run import main

import xarray as xr
import time
import os
import json

from pysd.py_backend import functions


In [ ]:
# default simulation parameters
run_params = {'time_step': 0.03125,
              'initial_time': 1995,
              'final_time': 2050}

# default configuration parameters
config = {'model_name': '',
          'time': time.strftime('%H:%M'),
          'silent': True,
          'verbose': False,
          'return_timestep': 1.0,  # results will be stored every year
          'scenario_sheet': 'BAU',
          'plot': False,
          'run_params': run_params,
          'update_params': {},
          'extDataFilePath':'',
          'fname': ''}

config = update_paths(config)


In [ ]:
# loading the model object
model = load_model(os.path.join(config['folder'], config['model_py']))

# Running simulations updating complex model components, xarrays, lookups, and constants

NOTE: When the whole function is replaced (as in the exemple right below), the values of the updated parameters will not be displayed on standard output

## Updating complex model components

In [ ]:
# replacing the whole original model function by a new one multiplied by a constant k
def new_energy_per_x_t():
    k = 2.0
    data = xr.DataArray(data=[
        model.components.liquids_per_x_hv() * saving_ratios_v().loc['hv liq'].values * k,
        model.components.liquids_per_x_hv() * saving_ratios_v().loc['hv hib'].values * k,
        model.components.liquids_per_x_hv() * saving_ratios_v().loc['hv gas'].values * k,
        model.components.liquids_per_x_lv() * saving_ratios_v().loc['lv liq'].values * k,
        model.components.liquids_per_x_lv() * saving_ratios_v().loc['lv elec'].values * k,
        model.components.liquids_per_x_lv() * saving_ratios_v().loc['lv hib'].values * k,
        model.components.liquids_per_x_lv() * saving_ratios_v().loc['lv gas'].values * k,
        model.components.liquids_per_x_bus() * saving_ratios_v().loc['bus liq'].values * k,
        model.components.liquids_per_x_bus() * saving_ratios_v().loc['bus elec'].values * k,
        model.components.liquids_per_x_bus() * saving_ratios_v().loc['bus hib'].values * k,
        model.components.liquids_per_x_bus() * saving_ratios_v().loc['bus gas'].values * k,
        model.components.energy_per_x_train() * 0.8 * k,
        model.components.energy_per_x_train() * 0.2 * k],
                        coords={'vehiclet': ['hv liq', 'hv hib', 'hv gas', 'lv liq', 'lv elec', 'lv hib', 'lv gas', 'bus liq', 'bus elec', 'bus hib', 'bus gas', 'train liq', 'train elec']},
                        dims=['vehiclet'])
    return data

# here we replace the component value by the new function
model.components.energy_per_x_t = new_energy_per_x_t

## Updating a function returning an xarray

In [ ]:
# giving new values to the leontief matrix

leontief_matrix_x1996x=xr.DataArray(
        data=[[
            10000000000000, 8.32659202e-03, 3.77221238e-01, 7.52160716e-02, 9.28578382e-02,
            2.06052968e-01, 2.85525958e-02, 9.94415558e-03, 2.21734806e-02, 2.92395654e-02,
            1.04966512e-02, 7.78537906e-03, 8.36514893e-03, 8.44549620e-03, 9.24354804e-03,
            3.89121816e-02, 5.30670320e-03, 1.77448005e-02, 6.40871057e-03, 9.07401780e-03,
            8.59456056e-03, 1.09667171e-01, 6.56618636e-03, 1.14147746e-02, 9.27584662e-03,
            9.95894183e-03, 4.84832243e-03, 4.51526682e-03, 3.37078235e-03, 7.28950337e-03,
            1.00822916e-02, 8.90046873e-03, 1.36371101e-02, 1.10712001e-02, 1.92964894e-02
        ], [
            2.36521725e-02, 1.13063555e+00, 2.61051069e-02, 2.77620299e-02, 2.32798820e-02,
            2.96108526e-02, 2.80583313e-02, 5.25149073e-01, 8.35126525e-02, 4.20722582e-02,
            1.28658091e-01, 9.86101641e-02, 3.71188365e-02, 2.97818406e-02, 3.28015029e-02,
            3.82664186e-02, 1.42457997e-01, 5.88467486e-02, 1.68244647e-02, 1.34081842e-02,
            1.15912435e-02, 1.94389090e-02, 4.65253373e-02, 4.63119380e-02, 4.45391050e-02,
            2.61817527e-02, 1.01747769e-02, 6.99250284e-03, 9.35709613e-03, 1.13304048e-02,
            1.75049791e-02, 1.35243252e-02, 1.64673177e-02, 1.82778877e-02, 7.35197697e-02
        ], [
            7.61004883e-02, 5.16755677e-03, 1.18984480e+00, 1.57263805e-02, 1.31615973e-01,
            2.01041379e-02, 1.18570724e-02, 6.07340918e-03, 2.14208310e-02, 1.37153385e-02,
            8.75045786e-03, 7.54115905e-03, 8.21245721e-03, 8.61502385e-03, 8.01807904e-03,
            1.09492980e-02, 4.95333459e-03, 7.74952879e-03, 6.99787695e-03, 1.04290314e-02,
            9.16160298e-03, 1.99304453e-01, 7.06144157e-03, 1.53629727e-02, 1.42904222e-02,
            1.14328528e-02, 6.19036350e-03, 5.76495921e-03, 2.30948326e-03, 8.68766190e-03,
            1.46174288e-02, 1.40197480e-02, 2.28849543e-02, 1.25472888e-02, 3.16076006e-03
        ], [
            6.48889338e-03, 5.14168534e-03, 7.70022571e-03, 1.40473261e+00, 9.94155158e-02,
            1.50563888e-02, 1.74722472e-02, 4.61735333e-03, 1.19629941e-02, 3.67305962e-02,
            1.24272641e-02, 7.58274965e-03, 9.53959703e-03, 9.46119075e-03, 1.85501817e-02,
            4.28521321e-02, 3.43704738e-03, 9.29826010e-03, 7.17011922e-03, 6.65185465e-03,
            7.31987016e-03, 8.23200872e-03, 5.80551649e-03, 8.73618769e-03, 5.60471726e-03,
            6.02371500e-03, 6.53023428e-03, 2.93801147e-03, 1.67790720e-03, 4.55977326e-03,
            7.48352708e-03, 2.81733373e-03, 8.01186040e-03, 1.03741925e-02, 1.40293576e-03
        ], [
            6.94736498e-04, 8.21390334e-04, 9.40973359e-04, 1.20558078e-02, 1.23425639e+00,
            2.38758119e-03, 2.74593756e-03, 7.13339394e-04, 1.08632409e-03, 2.33622305e-03,
            1.54227127e-03, 1.35772122e-03, 1.40866722e-03, 1.35145805e-03, 3.00140272e-03,
            5.81959430e-03, 6.47916232e-04, 9.73964566e-04, 1.17271712e-03, 9.77999235e-04,
            9.86566318e-04, 7.96940512e-04, 7.32499773e-04, 8.62805237e-04, 7.24331139e-04,
            6.86940437e-04, 1.21886908e-03, 3.76052122e-04, 1.94022907e-04, 5.72028830e-04,
            1.51618198e-03, 4.23087077e-04, 6.10627631e-04, 1.42532892e-03, 1.21130076e-04
        ], [
            4.36684012e-03, 5.26038452e-03, 6.10351413e-03, 6.44536438e-03, 7.55708364e-03,
            1.23525967e+00, 2.09893385e-02, 4.24654743e-03, 6.20586228e-03, 7.91826524e-03,
            1.32468443e-02, 8.67674671e-03, 8.72228659e-03, 7.25405572e-03, 1.02833140e-02,
            9.72816793e-02, 4.58714989e-03, 4.98055556e-02, 4.39541103e-03, 4.46318632e-03,
            4.09305084e-03, 6.21865591e-03, 4.75141640e-03, 4.77650314e-03, 4.00492206e-03,
            9.34749179e-03, 3.30780119e-03, 2.95585277e-03, 5.92554577e-03, 3.79128229e-03,
            5.04455486e-03, 3.75988948e-03, 3.84273015e-03, 7.22612785e-03, 1.33154546e-03
        ], [
            1.58106752e-02, 1.56386981e-02, 5.25627664e-02, 3.56904447e-02, 4.28486957e-02,
            3.62645506e-02, 1.32355451e+00, 1.57245350e-02, 4.94643252e-02, 4.93343376e-02,
            4.90305679e-02, 2.66605805e-02, 3.43410674e-02, 4.48000908e-02, 3.14405949e-02,
            5.00881674e-02, 1.52979233e-02, 2.63167433e-02, 2.44602092e-02, 3.05714148e-02,
            2.91645704e-02, 3.24582836e-02, 2.06767636e-02, 2.38297764e-02, 2.68900370e-02,
            3.66869667e-02, 2.59195130e-02, 3.41080835e-02, 9.85195647e-03, 4.25783449e-02,
            3.22451156e-02, 2.77227363e-02, 2.58276287e-02, 4.62473152e-02, 2.79947967e-03
        ], [
            2.20112315e-02, 2.37864261e-02, 1.87000655e-02, 1.97987169e-02, 1.74799691e-02,
            1.95259393e-02, 1.64164451e-02, 1.06390305e+00, 4.62912440e-02, 2.77426619e-02,
            3.17420729e-02, 2.45029561e-02, 1.62999064e-02, 1.50298078e-02, 1.53682653e-02,
            1.79416909e-02, 3.08681266e-02, 2.57439285e-02, 1.20883869e-02, 1.26236969e-02,
            1.04394737e-02, 1.30133064e-02, 6.25847506e-02, 7.04675053e-02, 6.97117249e-02,
            2.66093798e-02, 8.71281888e-03, 5.48774224e-03, 4.34004191e-03, 9.98800863e-03,
            1.39125151e-02, 7.26636670e-03, 1.10109037e-02, 1.32823770e-02, 2.27134904e-03
        ], [
            7.12781541e-02, 3.23406089e-02, 5.29518282e-02, 1.47961331e-01, 9.41342329e-02,
            6.99498946e-02, 7.63147022e-02, 3.92468787e-02, 1.32227052e+00, 3.10962492e-01,
            6.47097093e-02, 4.68069720e-02, 4.89483695e-02, 5.97174739e-02, 5.96307481e-02,
            8.14768013e-02, 2.01570011e-02, 4.00833985e-02, 2.92029646e-02, 1.42062516e-02,
            1.25081769e-02, 2.54264940e-02, 1.93312974e-02, 1.95381991e-02, 1.70697430e-02,
            1.81522545e-02, 1.13638138e-02, 7.91269977e-03, 7.09862413e-03, 1.55582564e-02,
            2.21792162e-02, 1.14451920e-02, 9.26443836e-02, 2.44391873e-02, 4.16390548e-03
        ], [
            1.24370112e-02, 1.72885491e-02, 2.88376537e-02, 2.32712215e-02, 5.85116954e-02,
            2.32369162e-02, 2.90517226e-02, 1.35038289e-02, 3.48988070e-02, 1.13733355e+00,
            2.36709474e-02, 2.20570335e-02, 4.69008346e-02, 4.91354667e-02, 6.92555293e-02,
            5.68846850e-02, 9.96793365e-03, 3.19787271e-02, 3.34088460e-02, 1.04169761e-02,
            1.03907042e-02, 1.40437829e-02, 2.11777072e-02, 1.29798210e-02, 1.21663586e-02,
            1.48988717e-02, 9.70643574e-03, 4.59649664e-03, 4.22187736e-03, 8.12650745e-03,
            1.00829610e-02, 4.68868171e-03, 1.29949896e-02, 1.01527154e-02, 1.90242475e-03
        ], [
            5.66489001e-03, 1.01944711e-02, 1.16504460e-02, 7.61343708e-03, 7.66038977e-03,
            1.78742761e-02, 6.42428628e-03, 8.76287956e-03, 1.42131462e-02, 1.42028848e-02,
            1.12397875e+00, 2.03391057e-02, 1.55640227e-02, 2.36629051e-02, 1.88925922e-02,
            1.54778577e-02, 8.14224143e-03, 9.83782688e-02, 8.40658917e-03, 4.47303787e-03,
            3.80525159e-03, 8.63204937e-03, 5.93372845e-03, 5.35217871e-03, 4.78226773e-03,
            6.35536158e-03, 4.75568123e-03, 2.60273163e-03, 7.81196334e-03, 4.00500062e-03,
            6.24141358e-03, 4.76697244e-03, 6.06659398e-03, 6.17852108e-03, 1.23875647e-03
        ], [
            2.42209236e-02, 6.24446239e-02, 4.69192583e-02, 3.49102103e-02, 4.48204250e-02,
            7.33819694e-02, 3.90320856e-02, 4.65737462e-02, 5.77711731e-02, 6.97492831e-02,
            8.07715425e-02, 1.49101445e+00, 3.02119657e-01, 1.75703119e-01, 2.49247407e-01,
            1.95602451e-01, 3.57780813e-02, 1.61908747e-01, 7.23544773e-02, 2.08935660e-02,
            1.80553163e-02, 2.62120682e-02, 3.63232670e-02, 4.64822551e-02, 3.91697989e-02,
            3.12808013e-02, 2.15065616e-02, 1.05574991e-02, 1.56769196e-02, 1.86418877e-02,
            2.77530588e-02, 1.32859073e-02, 2.05248552e-02, 2.34278778e-02, 6.48466094e-03
        ], [
            1.41212739e-02, 3.00014601e-02, 1.47403431e-02, 1.78879937e-02, 1.62327147e-02,
            2.13132188e-02, 1.74493173e-02, 2.19236372e-02, 2.00637127e-02, 2.39520001e-02,
            2.60502100e-02, 3.53941274e-02, 1.14899249e+00, 3.28270605e-02, 5.88493491e-02,
            2.33335432e-02, 1.72270903e-02, 3.03448137e-02, 3.74234317e-02, 8.80392497e-03,
            7.51583475e-03, 9.26351098e-03, 1.53869223e-02, 1.61728769e-02, 1.35260530e-02,
            1.22936044e-02, 7.89019675e-03, 4.64623928e-03, 4.61592126e-03, 7.92242170e-03,
            1.28628085e-02, 5.58634185e-03, 7.65238769e-03, 9.44516541e-03, 2.66297287e-03
        ], [
            1.11752146e-02, 2.22783784e-02, 1.82636799e-02, 2.24849204e-02, 2.12028429e-02,
            2.47702519e-02, 2.93456408e-02, 1.99193282e-02, 3.01209812e-02, 3.57654753e-02,
            2.88331004e-02, 4.59420621e-02, 1.32173316e-01, 1.35874304e+00, 1.15151430e-01,
            4.75502215e-02, 2.91951991e-02, 5.36580034e-02, 5.43995761e-02, 2.10649668e-02,
            1.86689123e-02, 1.64398460e-02, 2.25252672e-02, 2.24880198e-02, 2.21854770e-02,
            2.24664342e-02, 4.78720575e-02, 1.30968261e-02, 8.00896404e-03, 2.67090711e-02,
            2.70254382e-02, 1.33677419e-02, 3.37985480e-02, 2.13810943e-02, 3.06327350e-03
        ], [
            1.16781719e-02, 1.58578697e-02, 1.32095137e-02, 1.36658592e-02, 1.44100638e-02,
            1.51760344e-02, 1.13526237e-02, 1.35196308e-02, 1.29941073e-02, 1.49634855e-02,
            1.59612433e-02, 1.91466014e-02, 3.19632747e-02, 1.80526936e-02, 1.42240037e+00,
            1.67772695e-02, 1.01733620e-02, 1.56384578e-02, 1.33381693e-01, 1.31675101e-02,
            1.07225191e-02, 8.98112878e-03, 5.07009596e-02, 7.24831266e-02, 7.36943201e-02,
            2.50959630e-02, 9.27803369e-03, 5.19425816e-03, 3.18086439e-03, 9.60918107e-03,
            3.07844255e-02, 4.89134273e-03, 6.93402074e-03, 1.07322628e-02, 1.63966011e-03
        ], [
            2.49487283e-03, 3.49741895e-03, 3.84971662e-03, 1.03106098e-02, 7.62020548e-03,
            1.06092419e-02, 6.55441783e-03, 3.09923388e-03, 4.96818679e-03, 6.00274424e-03,
            6.69680342e-03, 1.29054124e-02, 9.07613762e-03, 7.76562911e-03, 1.17732848e-02,
            1.04136703e+00, 2.98630109e-03, 9.01295699e-03, 5.15948675e-03, 3.16673017e-03,
            3.11402638e-03, 4.99781452e-03, 3.53803508e-03, 3.78085685e-03, 3.71954763e-03,
            3.66290448e-03, 2.53002130e-03, 2.70497738e-03, 2.17417007e-03, 4.13447177e-03,
            5.86470143e-03, 4.79757401e-03, 4.72995350e-03, 8.36560746e-03, 4.12173269e-03
        ], [
            2.17211904e-02, 4.05202320e-02, 3.16730674e-02, 4.11586572e-02, 2.94931527e-02,
            3.77164742e-02, 4.43816069e-02, 4.03332449e-02, 5.68357241e-02, 5.01096678e-02,
            6.27474563e-02, 5.84721434e-02, 3.63296186e-02, 3.34308359e-02, 3.59345855e-02,
            3.51389106e-02, 1.11491704e+00, 2.69811063e-02, 2.42998815e-02, 1.56923037e-02,
            2.27464139e-02, 3.89042014e-02, 2.92040935e-02, 2.01675725e-02, 1.87341798e-02,
            2.76827000e-02, 1.69243981e-02, 1.21166394e-02, 1.37290338e-02, 1.58524035e-02,
            2.55629891e-02, 2.70253777e-02, 2.55635071e-02, 2.83475531e-02, 3.69174424e-03
        ], [
            8.97191308e-03, 1.49177163e-02, 1.10733195e-02, 1.11836836e-02, 1.06640501e-02,
            1.35483281e-02, 1.29121334e-02, 1.35709106e-02, 1.42931727e-02, 1.33564460e-02,
            1.77164648e-02, 1.56706787e-02, 1.27381094e-02, 1.26842336e-02, 1.20756272e-02,
            1.23910937e-02, 3.58750410e-02, 1.04499252e+00, 1.14007141e-02, 1.06063947e-02,
            1.14655107e-02, 1.36113372e-02, 1.72513519e-02, 1.47033020e-02, 1.14245509e-02,
            2.48820248e-02, 1.77384091e-02, 1.19705594e-02, 5.06262470e-02, 1.20103969e-02,
            2.69035838e-02, 1.90993894e-02, 1.37254219e-02, 2.11215051e-02, 4.09787074e-03
        ], [
            9.58115487e-03, 8.91650730e-03, 1.43701277e-02, 1.31764268e-02, 1.63199048e-02,
            1.41106634e-02, 1.13222265e-02, 1.00016903e-02, 1.40692465e-02, 1.44618089e-02,
            1.50160680e-02, 1.48383569e-02, 1.33697877e-02, 1.33304796e-02, 1.40510920e-02,
            1.39737745e-02, 1.43541371e-02, 1.48555008e-02, 1.01341363e+00, 8.09532671e-03,
            7.75353980e-03, 1.11356805e-02, 2.34538737e-02, 1.03220529e-02, 8.58302024e-03,
            1.32039302e-02, 8.17501257e-03, 5.70236050e-03, 3.95485412e-03, 9.92716152e-03,
            7.74312558e-03, 5.67648203e-03, 7.29620624e-03, 1.09574153e-02, 9.80087544e-04
        ], [
            5.93665927e-02, 4.92552199e-02, 1.07920694e-01, 1.03870207e-01, 1.30377154e-01,
            1.00161144e-01, 8.94843156e-02, 7.29871126e-02, 9.23759748e-02, 9.89408562e-02,
            8.10581337e-02, 9.00216049e-02, 9.73270881e-02, 1.03405122e-01, 1.02984212e-01,
            9.78036124e-02, 4.57566717e-02, 8.24062111e-02, 6.73699235e-02, 1.04975481e+00,
            2.76014748e-02, 7.76585957e-02, 4.61148614e-02, 4.80313060e-02, 4.48292131e-02,
            4.35437059e-02, 2.46139114e-02, 1.69850169e-02, 1.24931215e-02, 2.81219049e-02,
            3.87837708e-02, 2.13363699e-02, 4.84536151e-02, 3.36787329e-02, 5.60071950e-03
        ], [
            2.93677308e-02, 2.10886312e-02, 5.28982101e-02, 5.10653383e-02, 6.97797826e-02,
            4.21765239e-02, 3.61318822e-02, 2.89471277e-02, 4.27313284e-02, 4.43092348e-02,
            3.61270254e-02, 3.60689123e-02, 3.76703618e-02, 3.73979056e-02, 4.45165223e-02,
            4.66722118e-02, 1.86949037e-02, 4.54483133e-02, 3.04105412e-02, 1.70082222e-02,
            1.01561955e+00, 4.41255896e-02, 2.63833370e-02, 2.35920298e-02, 2.10551748e-02,
            2.15261439e-02, 1.80292713e-02, 1.08763938e-02, 8.33153575e-03, 1.79798262e-02,
            1.69842870e-02, 1.15075256e-02, 2.17465552e-02, 2.27478407e-02, 3.03134335e-03
        ], [
            5.87006517e-03, 9.19411572e-03, 1.15097346e-02, 1.29939256e-02, 1.33069235e-02,
            1.34736432e-02, 1.43567473e-02, 9.07894807e-03, 1.43221543e-02, 1.51166713e-02,
            1.40861242e-02, 1.45259430e-02, 1.60356481e-02, 1.69895741e-02, 1.35687782e-02,
            1.35036114e-02, 1.02491296e-02, 1.33483862e-02, 1.29559309e-02, 1.49624261e-02,
            1.08773227e-02, 1.01625786e+00, 1.45466846e-02, 1.71610118e-02, 3.24576452e-02,
            2.94540140e-02, 1.31563734e-02, 1.65452478e-02, 5.38874120e-03, 1.97430930e-02,
            1.43144130e-02, 9.90885442e-03, 1.35327041e-02, 1.74049782e-02, 1.03588425e-03
        ], [
            2.78268225e-02, 4.34258592e-02, 4.54417359e-02, 4.24062103e-02, 4.75187895e-02,
            4.91020083e-02, 3.88318894e-02, 6.32268352e-02, 4.38296983e-02, 4.16177918e-02,
            5.97644035e-02, 4.38437736e-02, 3.46303856e-02, 3.10895027e-02, 3.74084987e-02,
            4.15377451e-02, 4.18240526e-02, 3.95989230e-02, 2.72617969e-02, 2.87117321e-02,
            1.86834073e-02, 2.70184351e-02, 1.05744017e+00, 4.10321582e-02, 2.92495194e-02,
            4.75930098e-02, 1.81675634e-02, 1.38569437e-02, 6.94028073e-03, 1.57208273e-02,
            2.16448161e-02, 1.65959884e-02, 1.84204753e-02, 2.08862412e-02, 4.54195926e-03
        ], [
            5.19117046e-03, 6.87217592e-03, 7.95585015e-03, 7.04263692e-03, 8.40996891e-03,
            1.25323523e-02, 4.62867423e-03, 9.32617953e-03, 6.91930874e-03, 6.12384053e-03,
            9.34965886e-03, 8.95354634e-03, 5.46218479e-03, 4.95663496e-03, 5.97319126e-03,
            6.40827466e-03, 4.55431570e-03, 6.38534651e-03, 3.27379223e-03, 4.23890483e-03,
            2.66550069e-03, 4.43509555e-03, 5.95282574e-03, 1.11547883e+00, 6.10082166e-03,
            9.02001998e-03, 2.51072933e-03, 1.75203564e-03, 9.16921640e-04, 2.37880925e-03,
            3.04866479e-03, 2.61582574e-03, 2.21540209e-03, 2.57663027e-03, 6.85436039e-04
        ], [
            2.76958665e-03, 4.92062577e-03, 4.92796835e-03, 5.44052896e-03, 7.05445857e-03,
            4.77998039e-03, 5.84165003e-03, 4.51882750e-03, 5.84010454e-03, 5.70176512e-03,
            4.94198534e-03, 5.20572367e-03, 5.62219774e-03, 6.24114421e-03, 6.16008691e-03,
            5.20165682e-03, 3.15486242e-03, 4.40302315e-03, 4.82659631e-03, 6.83654639e-03,
            3.67619469e-03, 3.93383090e-03, 5.29495431e-03, 1.13856542e-02, 1.03824757e+00,
            2.48198931e-02, 7.57271366e-03, 5.92177649e-03, 1.61170443e-03, 6.93737047e-03,
            6.79264226e-03, 3.45130826e-03, 3.43401851e-03, 6.61508540e-03, 4.76071350e-04
        ], [
            1.12374476e-02, 1.65980031e-02, 1.96402992e-02, 1.68723852e-02, 1.98449138e-02,
            2.09842651e-02, 1.96339640e-02, 2.12470284e-02, 1.89038415e-02, 1.88174260e-02,
            2.37601155e-02, 2.01095673e-02, 1.77677975e-02, 1.69133650e-02, 1.86720178e-02,
            1.80247599e-02, 1.39389951e-02, 1.54924838e-02, 1.65299077e-02, 2.86787053e-02,
            1.57020348e-02, 1.82283037e-02, 5.06836583e-02, 1.85656547e-01, 1.20776566e-01,
            1.13829769e+00, 9.47668797e-03, 9.39179831e-03, 3.46910651e-03, 1.13862953e-02,
            1.12520285e-02, 6.71859342e-03, 9.44540856e-03, 1.25304611e-02, 1.62978050e-03
        ], [
            9.33691098e-03, 1.45261453e-02, 1.59161067e-02, 1.70140210e-02, 1.83062521e-02,
            1.64764606e-02, 2.48080431e-02, 1.41535808e-02, 2.02553559e-02, 1.89790948e-02,
            1.73306796e-02, 1.83150311e-02, 2.01480812e-02, 2.09092505e-02, 1.81008905e-02,
            1.96530464e-02, 1.36331113e-02, 2.03515439e-02, 2.12940626e-02, 2.78377899e-02,
            2.64174598e-02, 2.16618073e-02, 2.46344808e-02, 3.79330110e-02, 4.46292775e-02,
            3.56904370e-02, 1.09442061e+00, 3.42223123e-02, 8.91279934e-03, 3.06601618e-02,
            2.54264826e-02, 1.27779641e-02, 1.87970407e-02, 3.14440680e-02, 1.64952217e-03
        ], [
            4.53859959e-02, 6.35674434e-02, 5.86577378e-02, 6.75338041e-02, 6.59375285e-02,
            6.56467993e-02, 6.26532342e-02, 5.79038628e-02, 6.61416561e-02, 6.37873380e-02,
            6.54957466e-02, 6.82078619e-02, 6.48297434e-02, 6.28211232e-02, 6.19955060e-02,
            6.52369877e-02, 5.36732742e-02, 6.02577730e-02, 5.80607429e-02, 7.21142437e-02,
            6.14151318e-02, 5.76165418e-02, 7.49913540e-02, 8.72187058e-02, 7.67371534e-02,
            6.30327414e-02, 4.68914149e-02, 1.22167500e+00, 7.02905284e-02, 6.45728991e-02,
            4.73952143e-02, 2.85347743e-02, 4.81668241e-02, 6.27031493e-02, 1.01523717e-02
        ], [
            1.70105852e-02, 1.64002017e-02, 2.20121918e-02, 2.30832392e-02, 2.55902838e-02,
            2.23298794e-02, 2.56888142e-02, 1.67592389e-02, 2.21148882e-02, 2.35853589e-02,
            2.09245473e-02, 2.22428894e-02, 2.36498151e-02, 2.34028497e-02, 2.25521124e-02,
            2.59809365e-02, 1.67568776e-02, 2.88932198e-02, 3.77717897e-02, 4.07483141e-02,
            5.33149858e-02, 4.33757173e-02, 2.37105814e-02, 4.85924943e-02, 3.34442324e-02,
            3.74772424e-02, 2.23922623e-02, 3.91759974e-02, 1.03730688e+00, 3.65893621e-02,
            2.13513373e-02, 2.41506545e-02, 4.10244349e-02, 4.17115125e-02, 2.48926377e-03
        ], [
            4.73707499e-02, 7.11564215e-02, 9.51780515e-02, 9.26142264e-02, 9.44545709e-02,
            8.04353967e-02, 1.25739173e-01, 7.04320405e-02, 1.32969188e-01, 1.07544261e-01,
            9.15001520e-02, 9.22898400e-02, 1.12502451e-01, 1.37673371e-01, 1.16295767e-01,
            1.00663368e-01, 7.72469760e-02, 1.11162226e-01, 9.22675163e-02, 9.87935520e-02,
            9.93659743e-02, 8.20798741e-02, 8.49152077e-02, 1.22537660e-01, 1.55205870e-01,
            1.20516687e-01, 1.03562556e-01, 1.36299977e-01, 5.82954726e-02, 1.17526955e+00,
            9.33261872e-02, 4.89112911e-02, 9.19520348e-02, 1.10984361e-01, 7.48811386e-03
        ], [
            3.75609017e-03, 6.05567550e-03, 5.33074870e-03, 4.61625365e-03, 4.48070658e-03,
            6.93185857e-03, 6.90503099e-03, 5.15321449e-03, 7.11869963e-03, 6.14415430e-03,
            5.94662001e-03, 6.47456165e-03, 4.94559296e-03, 5.19653632e-03, 5.09264707e-03,
            6.12581271e-03, 8.16458391e-03, 4.73828678e-03, 4.69353327e-03, 3.86990263e-03,
            5.46841001e-03, 6.38274370e-03, 5.64729305e-03, 1.30600089e-02, 1.22563668e-02,
            6.91786445e-03, 4.51796485e-03, 3.55539884e-03, 4.72706472e-03, 4.97248703e-03,
            1.00972721e+00, 5.20412233e-03, 7.48232305e-03, 1.04794961e-02, 5.45051936e-04
        ], [
            1.53096123e-03, 1.22847263e-03, 1.59467640e-03, 1.53042602e-03, 1.60375554e-03,
            1.38343993e-03, 1.80034932e-03, 1.42187577e-03, 2.11935021e-03, 1.73657004e-03,
            1.56042409e-03, 1.64750928e-03, 2.03549444e-03, 2.33187207e-03, 1.83637930e-03,
            1.47123713e-03, 1.82752779e-03, 1.53930950e-03, 1.28367760e-03, 1.66696739e-03,
            2.58279399e-03, 1.32584575e-03, 1.91612182e-03, 1.52998291e-03, 2.91354915e-03,
            2.12428979e-03, 2.38497998e-03, 2.12466181e-03, 5.86339779e-04, 3.00767356e-03,
            5.17186705e-03, 1.00686285e+00, 1.80950013e-03, 2.28675933e-03, 1.35016301e-04
        ], [
            2.79177799e-03, 8.81247867e-04, 1.76697414e-03, 1.32133761e-03, 1.35250233e-03,
            1.44467599e-03, 1.18988785e-03, 8.65198022e-04, 1.23032466e-03, 1.19687224e-03,
            1.12182063e-03, 1.12924160e-03, 1.29597065e-03, 1.25501765e-03, 1.14076503e-03,
            1.30028950e-03, 9.23851404e-04, 1.21332060e-03, 1.04989223e-03, 1.09990068e-03,
            8.65302268e-04, 1.79191993e-03, 1.22025549e-03, 1.75071850e-03, 1.27692620e-03,
            1.33009264e-03, 1.23196667e-03, 1.08321698e-03, 3.60844578e-04, 1.29816155e-03,
            6.77230039e-03, 1.74588266e-03, 1.02326055e+00, 2.29201583e-03, 1.21566362e-04
        ], [
            9.74263658e-03, 1.50785046e-02, 1.92441004e-02, 1.92417218e-02, 2.13349738e-02,
            1.82186492e-02, 2.94912150e-02, 1.54075623e-02, 2.38128194e-02, 2.10069309e-02,
            1.88098926e-02, 2.18577734e-02, 2.15708897e-02, 2.18557871e-02, 2.00835171e-02,
            2.08659665e-02, 1.55580608e-02, 1.95458276e-02, 2.07285716e-02, 1.91676062e-02,
            1.98120387e-02, 2.52881145e-02, 2.01620866e-02, 2.16205071e-02, 2.38277603e-02,
            2.43873652e-02, 2.82599078e-02, 2.22857085e-02, 1.32456747e-02, 4.23728685e-02,
            2.49740428e-02, 1.61663126e-02, 2.55940348e-02, 1.09457767e+00, 2.02732839e-03
        ], [
            4.80985248e-05, 5.83062239e-05, 9.41535010e-05, 1.10374700e-04, 1.02333785e-04,
            9.98930715e-05, 1.04199249e-04, 6.73697321e-05, 9.72789540e-05, 1.03461985e-04,
            8.57559870e-05, 1.03574045e-04, 9.99745458e-05, 8.81312500e-05, 1.21661532e-04,
            1.03802489e-04, 6.09055197e-05, 1.49703133e-04, 1.93925924e-04, 9.18345830e-05,
            1.08285004e-04, 1.26712162e-04, 2.19469169e-04, 9.98065386e-05, 1.00857829e-04,
            9.60586634e-05, 1.28294613e-04, 1.19029245e-04, 3.16445850e-04, 1.25758585e-04,
            1.32734920e-04, 8.34228095e-05, 1.77193186e-04, 1.82126670e-04, 1.00003186e+00
        ]],
        coords={
            'sectors': [
                'agriculture hunting forestry and fishing', 'mining and quarrying',
                'food beverages and tobacco', 'textiles and textile products',
                'leather leather and footwear', 'wood and products of woood and cork',
                'pulp paper printing and publishing', 'coke refined petroleum and nuclear fuel',
                'chemicals and chemical products', 'rubber and plastics',
                'other non metalic mineral', 'basic metals and fabricated metal', 'machinery nec',
                'electrical and optical equipment', 'transport equipment',
                'manufacturing nec recycling', 'electricity gas and water supply', 'construction',
                'sale maintenance and repair of motor vehicles anda motorcycles retail sale of fuel',
                'wholesale trade and commissions trade except of motor vehicles and motorcycles',
                'retail trade except of motor vehicles and motorcycles repair of household goods',
                'hotels and restaurants', 'inland transport', 'water transport', 'air transport',
                'other supporting and auxiliary transport activities activities of travel agencies',
                'post and telecommunications', 'financial intermedation', 'real estate activities',
                'renting od meq and other business activities',
                'public admin and defence compulsory social security', 'education',
                'health and social work', 'other community social and persona services',
                'private households with employed persons'
            ],
            'sectors1': [
                'agriculture hunting forestry and fishing', 'mining and quarrying',
                'food beverages and tobacco', 'textiles and textile products',
                'leather leather and footwear', 'wood and products of woood and cork',
                'pulp paper printing and publishing', 'coke refined petroleum and nuclear fuel',
                'chemicals and chemical products', 'rubber and plastics',
                'other non metalic mineral', 'basic metals and fabricated metal', 'machinery nec',
                'electrical and optical equipment', 'transport equipment',
                'manufacturing nec recycling', 'electricity gas and water supply', 'construction',
                'sale maintenance and repair of motor vehicles anda motorcycles retail sale of fuel',
                'wholesale trade and commissions trade except of motor vehicles and motorcycles',
                'retail trade except of motor vehicles and motorcycles repair of household goods',
                'hotels and restaurants', 'inland transport', 'water transport', 'air transport',
                'other supporting and auxiliary transport activities activities of travel agencies',
                'post and telecommunications', 'financial intermedation', 'real estate activities',
                'renting od meq and other business activities',
                'public admin and defence compulsory social security', 'education',
                'health and social work', 'other community social and persona services',
                'private households with employed persons'
            ]
        },
        dims=['sectors', 'sectors1'])





## Updating a function returning a lookup

### First way (with string format)

In [ ]:
# changing the values of a lookup
historic_water_use_by_type_sectors_subelectricity_gas_and_water_supply_subgreen_water = "functions.lookup(x, [1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])"

### Second way (replacing the whole function)

In [ ]:
def new_table_max_extraction_unconv_oil_bg_mohr15(x):

    return functions.lookup(x, [
        23.689092215699986, 19.60434318569999, 16.034516965699993, 12.949607615699989,
        10.246755295699984, 7.903732325699991, 5.956996261699999, 4.4042328447000045,
        3.222744901699997, 1.9837517516999839, 0.0, 0.03395225589999917, 0.0
    ], [
        2.23090393, 1.97989792, 1.72180278, 1.48436333,
        1.30811635, 1.12598014, 0.9235706239999999, 0.7276802210000001,
        0.5480696700000001, 0.354303314, 0.0, 0.028279365999999997, 0.0
    ])
model.components.table_max_extraction_unconv_oil_bg_mohr15 = new_table_max_extraction_unconv_oil_bg_mohr15

In [ ]:
#changing the values of a constant
land_productivity_biofuels_marg_ej_mha = 0.4

In [ ]:
# running the simulation with the updated parameters
main(model, config, run_params, energy_per_x_t=energy_per_x_t,
     leontief_matrix_x1996x=leontief_matrix_x1996x, 
     historic_water_use_by_type_sectors_subelectricity_gas_and_water_supply_subgreen_water=historic_water_use_by_type_sectors_subelectricity_gas_and_water_supply_subgreen_water,
    land_productivity_biofuels_marg_ej_mha=land_productivity_biofuels_marg_ej_mha)

# Running several Scenarios in batch (not in parallel)

In [ ]:
# define the names of the scenarios to run (names of excel sheets in the inputs file)
scenarios = ['BAU', 'OLT']

for scen in scenarios:
    config.update({'scenario_sheet': scen, 'update_params': {}})
    main(model, config, run_params)

# Storing all variables defined as xarray in a json file

In [ ]:
xrrys = {}
lookps = {}

for component in [element for element in dir(model.components) if not element.startswith("_")]:
    try:
        value = getattr(model.components, component)()
    except:
        continue
    else:
        if isinstance(value, xr.DataArray):
            xrrys.update({component: value.to_dict()})

j = json.dumps(xrrys)
f = open("xarrays.json", "w")
f.write(j)
f.close()